<a href="https://colab.research.google.com/github/yashshinde0607/DS_LAB_SE_B_67/blob/main/DSLAB2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#Program for BFS
from collections import deque

def bfs(graph, start_node):
    visited = set()
    queue = deque([start_node])
    visited.add(start_node)

    print(f"BFS Traversal starting from node {start_node}:")
    path = []
    while queue:
        current_node = queue.popleft()
        path.append(current_node)

        for neighbor in graph[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    print(" ".join(path))
    print("\n")

# New Example Graph for BFS (Adjacency List)
new_bfs_graph = {
    'S': ['A', 'B'],
    'A': ['S', 'C', 'D'],
    'B': ['S', 'E'],
    'C': ['A'],
    'D': ['A', 'G'],
    'E': ['B', 'F'],
    'F': ['E', 'G'],
    'G': ['D', 'F']
}

# Perform BFS traversal on the new graph
bfs(new_bfs_graph, 'S')


BFS Traversal starting from node S:
S A B C D E G F




In [9]:
#Program for DFS
def dfs(graph, start_node, visited=None):
    if visited is None:
        visited = set()
        print(f"DFS Traversal starting from node {start_node}:")

    visited.add(start_node)
    print(start_node, end=" ")

    for neighbor in graph[start_node]:
        if neighbor not in visited:
            dfs(graph, neighbor, visited)

# New Example Graph for DFS (Adjacency List)
new_dfs_graph = {
    '1': ['2', '7'],
    '2': ['3', '6'],
    '3': ['4'],
    '4': ['5'],
    '5': [],
    '6': [],
    '7': ['8'],
    '8': ['9'],
    '9': []
}

# Perform DFS traversal on the new graph
dfs(new_dfs_graph, '1')


DFS Traversal starting from node 1:
1 2 3 4 5 6 7 8 9 

In [10]:
#program for A* Algorithm
import heapq

def a_star_search(graph, start, goal, heuristic):
    # g_score is the cost from start to current node
    g_score = {node: float('inf') for node in graph}
    g_score[start] = 0

    # f_score is g_score + heuristic estimate from current to goal
    f_score = {node: float('inf') for node in graph}
    f_score[start] = heuristic[start]

    # open_set stores (f_score, node) tuples, prioritized by f_score
    open_set = [(f_score[start], start)]

    # came_from records the most efficient previous step in the path to the current node
    came_from = {}

    while open_set:
        current_f_score, current_node = heapq.heappop(open_set)

        if current_node == goal:
            path = []
            while current_node in came_from:
                path.append(current_node)
                current_node = came_from[current_node]
            path.append(start)
            return path[::-1] # Reverse path to get it from start to goal

        # If we found a path to current_node with a higher f_score than already processed, skip it
        if current_f_score > f_score[current_node]:
            continue

        for neighbor, weight in graph[current_node].items():
            tentative_g_score = g_score[current_node] + weight

            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current_node
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic[neighbor]
                heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None # No path found

# Example Graph (Adjacency List with weights)
# Format: node: {neighbor: weight}
example_graph = {
    'A': {'B': 1, 'C': 3},
    'B': {'D': 2, 'E': 4},
    'C': {'F': 5},
    'D': {'G': 3},
    'E': {'G': 1},
    'F': {'G': 1},
    'G': {}
}

# Example Heuristic Function (straight-line distance to goal 'G')
# In a real-world scenario, this would be calculated based on geographical coordinates or similar.
# Here, it's a simplified estimate.
example_heuristic = {
    'A': 7,
    'B': 6,
    'C': 4,
    'D': 3,
    'E': 2,
    'F': 1,
    'G': 0
}

start_node = 'A'
goal_node = 'G'

print(f"Performing A* search from {start_node} to {goal_node}:")
path = a_star_search(example_graph, start_node, goal_node, example_heuristic)

if path:
    print("Path found:", ' -> '.join(path))
    # Calculate total cost of the path
    total_cost = 0
    for i in range(len(path) - 1):
        total_cost += example_graph[path[i]][path[i+1]]
    print("Total cost:", total_cost)
else:
    print("No path found!")


Performing A* search from A to G:
Path found: A -> B -> D -> G
Total cost: 6


In [11]:
#program for minimax Algorithm
import math

# --- Helper functions for a simplified Tic-Tac-Toe-like game ---

# Represents the board as a list. None means empty, 'X' for player, 'O' for opponent.
# For simplicity, let's make it a 3x3 board flattened to a list of 9 elements.
# Player 'X' is the maximizing player, Player 'O' is the minimizing player.

def create_board():
    return [None] * 9

def print_board(board):
    for i in range(0, 9, 3):
        row = [cell if cell is not None else str(j) for j, cell in enumerate(board[i:i+3])]
        print(" | ".join(row))
        if i < 6:
            print("---------")
    print("\n")

def get_available_moves(board):
    return [i for i, cell in enumerate(board) if cell is None]

def make_move(board, move, player):
    new_board = list(board) # Create a copy to avoid modifying original board
    new_board[move] = player
    return new_board

def check_win(board, player):
    win_conditions = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8), # rows
        (0, 3, 6), (1, 4, 7), (2, 5, 8), # columns
        (0, 4, 8), (2, 4, 6)            # diagonals
    ]
    for condition in win_conditions:
        if all(board[i] == player for i in condition):
            return True
    return False

def is_game_over(board):
    return check_win(board, 'X') or check_win(board, 'O') or len(get_available_moves(board)) == 0

def evaluate(board):
    if check_win(board, 'X'):
        return 1  # 'X' wins (maximizing player)
    elif check_win(board, 'O'):
        return -1 # 'O' wins (minimizing player)
    else:
        return 0  # Draw or game in progress

# --- Minimax Algorithm ---

def minimax(board, depth, is_maximizing_player):
    if is_game_over(board):
        return evaluate(board)

    if is_maximizing_player:
        best_value = -math.inf
        for move in get_available_moves(board):
            new_board = make_move(board, move, 'X')
            value = minimax(new_board, depth + 1, False)
            best_value = max(best_value, value)
        return best_value
    else:
        best_value = math.inf
        for move in get_available_moves(board):
            new_board = make_move(board, move, 'O')
            value = minimax(new_board, depth + 1, True)
            best_value = min(best_value, value)
        return best_value

def find_best_move(board):
    best_value = -math.inf
    best_move = -1

    for move in get_available_moves(board):
        new_board = make_move(board, move, 'X') # Assume 'X' is the AI player
        move_value = minimax(new_board, 0, False) # After AI's move, it's opponent's turn (minimizing)

        if move_value > best_value:
            best_value = move_value
            best_move = move

    return best_move

# --- Demonstration ---

current_board = create_board()

# Example Scenario 1: AI to make the first move
print("Initial Board (Empty):")
print_board(current_board)

print("AI (X) calculates its best first move...")
ai_move = find_best_move(current_board)
current_board = make_move(current_board, ai_move, 'X')
print(f"AI (X) plays at position {ai_move}:")
print_board(current_board)

# Example Scenario 2: A more complex scenario where AI needs to block or win
print("\n--- Another Scenario ---")
# Let's set up a board where AI (X) can win or block
scenario_board = create_board()
scenario_board[0] = 'X'
scenario_board[1] = 'O'
scenario_board[3] = 'X'
scenario_board[4] = 'O'

print("Current Board (AI's turn, X to play):")
print_board(scenario_board)

print("AI (X) calculates its best move...")
ai_move_2 = find_best_move(scenario_board)
scenario_board = make_move(scenario_board, ai_move_2, 'X')
print(f"AI (X) plays at position {ai_move_2}:")
print_board(scenario_board)



Initial Board (Empty):
0 | 1 | 2
---------
0 | 1 | 2
---------
0 | 1 | 2


AI (X) calculates its best first move...
AI (X) plays at position 0:
X | 1 | 2
---------
0 | 1 | 2
---------
0 | 1 | 2



--- Another Scenario ---
Current Board (AI's turn, X to play):
X | O | 2
---------
X | O | 2
---------
0 | 1 | 2


AI (X) calculates its best move...
AI (X) plays at position 6:
X | O | 2
---------
X | O | 2
---------
X | 1 | 2


